# Get data for all sectors: OCHLV + Indicators

In [1]:
%load_ext rich

In [2]:
import pandas as pd
import numpy as np

import urllib.request as request
from dotenv import load_dotenv
import os
import time

from rich import print
from rich.progress import Progress, SpinnerColumn, TimeElapsedColumn

In [3]:
load_dotenv()
API_KEY = os.getenv("ALPHA_VANTAGE_API")

In [4]:
sectors = ["XLC", "XLY", "XLP", "XLE", "XLF", "XLV", "XLI", "XLB", "XLRE", "XLK", "XLU"]
sp500 = "SPY"

start_date = "1999-12-31"

In [5]:
indicators = {
    "open-SMA-5" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'open')],
    "open-SMA-10" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'open')],
    "open-SMA-20" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "open-SMA-50" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'open')],
    "open-SMA-100" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'open')],
    "open-SMA-200" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'open')],
    "open-SMA-500" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'open')],
    "close-SMA-5" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'close')],
    "close-SMA-10" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'close')],
    "close-SMA-20" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "close-SMA-50" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'close')],
    "close-SMA-100" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'close')],
    "close-SMA-200" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'close')],
    "close-SMA-500" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'close')],
    "high-SMA-5" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'high')],
    "high-SMA-10" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'high')],
    "high-SMA-20" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "high-SMA-50" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'high')],
    "high-SMA-100" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'high')],
    "high-SMA-200" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'high')],
    "high-SMA-500" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'high')],
    "low-SMA-5" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'low')],
    "low-SMA-10" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'low')],
    "low-SMA-20" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "low-SMA-50" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'low')],
    "low-SMA-100" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'low')],
    "low-SMA-200" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'low')],
    "low-SMA-500" : [('function', 'SMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'low')],
    "open-EMA-12" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 12), ('series_type', 'open')],
    "open-EMA-26" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 26), ('series_type', 'open')],
    "open-EMA-50" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'open')],
    "open-EMA-200" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'open')],
    "close-EMA-12" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 12), ('series_type', 'close')],
    "close-EMA-26" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 26), ('series_type', 'close')],
    "close-EMA-50" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'close')],
    "close-EMA-200" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'close')],
    "high-EMA-12" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 12), ('series_type', 'high')],
    "high-EMA-26" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 26), ('series_type', 'high')],
    "high-EMA-50" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'high')],
    "high-EMA-200" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'high')],
    "low-EMA-12" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 12), ('series_type', 'low')],
    "low-EMA-26" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 26), ('series_type', 'low')],
    "low-EMA-50" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'low')],
    "low-EMA-200" : [('function', 'EMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'low')],
    "open-WMA-5" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'open')],
    "open-WMA-10" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'open')],
    "open-WMA-20" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "open-WMA-50" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'open')],
    "open-WMA-100" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'open')],
    "open-WMA-200" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'open')],
    "open-WMA-500" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'open')],
    "close-WMA-5" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'close')],
    "close-WMA-10" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'close')],
    "close-WMA-20" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "close-WMA-50" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'close')],
    "close-WMA-100" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'close')],
    "close-WMA-200" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'close')],
    "close-WMA-500" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'close')],
    "high-WMA-5" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'high')],
    "high-WMA-10" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'high')],
    "high-WMA-20" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "high-WMA-50" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'high')],
    "high-WMA-100" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'high')],
    "high-WMA-200" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'high')],
    "high-WMA-500" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'high')],
    "low-WMA-5" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'low')],
    "low-WMA-10" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'low')],
    "low-WMA-20" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "low-WMA-50" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'low')],
    "low-WMA-100" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'low')],
    "low-WMA-200" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'low')],
    "low-WMA-500" : [('function', 'WMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'low')],
    "open-DEMA-20" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "open-DEMA-50" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'open')],
    "open-DEMA-100" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'open')],
    "open-DEMA-200" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'open')],
    "close-DEMA-20" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "close-DEMA-50" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'close')],
    "close-DEMA-100" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'close')],
    "close-DEMA-200" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'close')],
    "high-DEMA-20" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "high-DEMA-50" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'high')],
    "high-DEMA-100" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'high')],
    "high-DEMA-200" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'high')],
    "low-DEMA-20" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "low-DEMA-50" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'low')],
    "low-DEMA-100" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'low')],
    "low-DEMA-200" : [('function', 'DEMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'low')],
    "open-TEMA-10" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'open')],
    "open-TEMA-20" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "open-TEMA-50" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'open')],
    "open-TEMA-100" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'open')],
    "close-TEMA-10" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'close')],
    "close-TEMA-20" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "close-TEMA-50" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'close')],
    "close-TEMA-100" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'close')],
    "high-TEMA-10" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'high')],
    "high-TEMA-20" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "high-TEMA-50" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'high')],
    "high-TEMA-100" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'high')],
    "low-TEMA-10" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'low')],
    "low-TEMA-20" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "low-TEMA-50" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'low')],
    "low-TEMA-100" : [('function', 'TEMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'low')],
    "open-TRIMA-5" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'open')],
    "open-TRIMA-10" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'open')],
    "open-TRIMA-20" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "open-TRIMA-50" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'open')],
    "open-TRIMA-100" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'open')],
    "open-TRIMA-200" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'open')],
    "open-TRIMA-500" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'open')],
    "close-TRIMA-5" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'close')],
    "close-TRIMA-10" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'close')],
    "close-TRIMA-20" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "close-TRIMA-50" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'close')],
    "close-TRIMA-100" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'close')],
    "close-TRIMA-200" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'close')],
    "close-TRIMA-500" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'close')],
    "high-TRIMA-5" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'high')],
    "high-TRIMA-10" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'high')],
    "high-TRIMA-20" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "high-TRIMA-50" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'high')],
    "high-TRIMA-100" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'high')],
    "high-TRIMA-200" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'high')],
    "high-TRIMA-500" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'high')],
    "low-TRIMA-5" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'low')],
    "low-TRIMA-10" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'low')],
    "low-TRIMA-20" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "low-TRIMA-50" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'low')],
    "low-TRIMA-100" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'low')],
    "low-TRIMA-200" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'low')],
    "low-TRIMA-500" : [('function', 'TRIMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'low')],
    "open-KAMA-5" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'open')],
    "open-KAMA-10" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'open')],
    "open-KAMA-20" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "open-KAMA-50" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'open')],
    "open-KAMA-100" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'open')],
    "open-KAMA-200" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'open')],
    "open-KAMA-500" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'open')],
    "close-KAMA-5" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'close')],
    "close-KAMA-10" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'close')],
    "close-KAMA-20" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "close-KAMA-50" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'close')],
    "close-KAMA-100" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'close')],
    "close-KAMA-200" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'close')],
    "close-KAMA-500" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'close')],
    "high-KAMA-5" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'high')],
    "high-KAMA-10" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'high')],
    "high-KAMA-20" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "high-KAMA-50" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'high')],
    "high-KAMA-100" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'high')],
    "high-KAMA-200" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'high')],
    "high-KAMA-500" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'high')],
    "low-KAMA-5" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'low')],
    "low-KAMA-10" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'low')],
    "low-KAMA-20" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "low-KAMA-50" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'low')],
    "low-KAMA-100" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 100), ('series_type', 'low')],
    "low-KAMA-200" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'low')],
    "low-KAMA-500" : [('function', 'KAMA'), ('interval', 'daily'), ('time_period', 500), ('series_type', 'low')],
    "open-MAMA" : [('function', 'MAMA'), ('interval', 'daily'), ('series_type', 'open'), ('slow_limit', 0.05), ('fast_limit', 0.5)],
    "open-MAMA" : [('function', 'MAMA'), ('interval', 'daily'), ('series_type', 'open'), ('slow_limit', 0.05), ('fast_limit', 0.3)],
    "close-MAMA" : [('function', 'MAMA'), ('interval', 'daily'), ('series_type', 'close'), ('slow_limit', 0.05), ('fast_limit', 0.5)],
    "close-MAMA" : [('function', 'MAMA'), ('interval', 'daily'), ('series_type', 'close'), ('slow_limit', 0.05), ('fast_limit', 0.3)],
    "high-MAMA" : [('function', 'MAMA'), ('interval', 'daily'), ('series_type', 'high'), ('slow_limit', 0.05), ('fast_limit', 0.5)],
    "high-MAMA" : [('function', 'MAMA'), ('interval', 'daily'), ('series_type', 'high'), ('slow_limit', 0.05), ('fast_limit', 0.3)],
    "low-MAMA" : [('function', 'MAMA'), ('interval', 'daily'), ('series_type', 'low'), ('slow_limit', 0.05), ('fast_limit', 0.5)],
    "low-MAMA" : [('function', 'MAMA'), ('interval', 'daily'), ('series_type', 'low'), ('slow_limit', 0.05), ('fast_limit', 0.3)],
    "open-T3-20" : [('function', 'T3'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "open-T3-50" : [('function', 'T3'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'open')],
    "open-T3-200" : [('function', 'T3'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'open')],
    "close-T3-20" : [('function', 'T3'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "close-T3-50" : [('function', 'T3'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'close')],
    "close-T3-200" : [('function', 'T3'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'close')],
    "high-T3-20" : [('function', 'T3'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "high-T3-50" : [('function', 'T3'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'high')],
    "high-T3-200" : [('function', 'T3'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'high')],
    "low-T3-20" : [('function', 'T3'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "low-T3-50" : [('function', 'T3'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'low')],
    "low-T3-200" : [('function', 'T3'), ('interval', 'daily'), ('time_period', 200), ('series_type', 'low')],
    "open-MACD" : [('function', 'MACD'), ('interval', 'daily'), ('series_type', 'open'), ('fast-period',12), ('slow-period',26), ('signal-period', 9)],
    "open-MACD" : [('function', 'MACD'), ('interval', 'daily'), ('series_type', 'open'), ('fast-period',5), ('slow-period',13), ('signal-period', 5)],
    "close-MACD" : [('function', 'MACD'), ('interval', 'daily'), ('series_type', 'close'), ('fast-period',12), ('slow-period',26), ('signal-period', 9)],
    "close-MACD" : [('function', 'MACD'), ('interval', 'daily'), ('series_type', 'close'), ('fast-period',5), ('slow-period',13), ('signal-period', 5)],
    "high-MACD" : [('function', 'MACD'), ('interval', 'daily'), ('series_type', 'high'), ('fast-period',12), ('slow-period',26), ('signal-period', 9)],
    "high-MACD" : [('function', 'MACD'), ('interval', 'daily'), ('series_type', 'high'), ('fast-period',5), ('slow-period',13), ('signal-period', 5)],
    "low-MACD" : [('function', 'MACD'), ('interval', 'daily'), ('series_type', 'low'), ('fast-period',12), ('slow-period',26), ('signal-period', 9)],
    "low-MACD" : [('function', 'MACD'), ('interval', 'daily'), ('series_type', 'low'), ('fast-period',5), ('slow-period',13), ('signal-period', 5)],
    "STOCH": [('function', 'STOCH'), ('interval', 'daily')],
    "open-RSI-10" : [('function', 'RSI'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'open')],
    "open-RSI-15" : [('function', 'RSI'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'open')],
    "open-RSI-20" : [('function', 'RSI'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "close-RSI-10" : [('function', 'RSI'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'close')],
    "close-RSI-15" : [('function', 'RSI'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'close')],
    "close-RSI-20" : [('function', 'RSI'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "high-RSI-10" : [('function', 'RSI'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'high')],
    "high-RSI-15" : [('function', 'RSI'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'high')],
    "high-RSI-20" : [('function', 'RSI'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "low-RSI-10" : [('function', 'RSI'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'low')],
    "low-RSI-15" : [('function', 'RSI'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'low')],
    "low-RSI-20" : [('function', 'RSI'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "open-STOCHRSI-10" : [('function', 'STOCHRSI'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'open')],
    "open-STOCHRSI-15" : [('function', 'STOCHRSI'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'open')],
    "open-STOCHRSI-20" : [('function', 'STOCHRSI'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "close-STOCHRSI-10" : [('function', 'STOCHRSI'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'close')],
    "close-STOCHRSI-15" : [('function', 'STOCHRSI'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'close')],
    "close-STOCHRSI-20" : [('function', 'STOCHRSI'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "high-STOCHRSI-10" : [('function', 'STOCHRSI'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'high')],
    "high-STOCHRSI-15" : [('function', 'STOCHRSI'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'high')],
    "high-STOCHRSI-20" : [('function', 'STOCHRSI'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "low-STOCHRSI-10" : [('function', 'STOCHRSI'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'low')],
    "low-STOCHRSI-15" : [('function', 'STOCHRSI'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'low')],
    "low-STOCHRSI-20" : [('function', 'STOCHRSI'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "WILLR-10": [('function', 'WILLR'), ('interval', 'daily'), ('time_period', 10)],
    "WILLR-20": [('function', 'WILLR'), ('interval', 'daily'), ('time_period', 20)],
    "WILLR-30": [('function', 'WILLR'), ('interval', 'daily'), ('time_period', 30)],
    "ADX-10": [('function', 'ADX'), ('interval', 'daily'), ('time_period', 10)],
    "ADX-20": [('function', 'ADX'), ('interval', 'daily'), ('time_period', 20)],
    "ADX-30": [('function', 'ADX'), ('interval', 'daily'), ('time_period', 30)],
    "ADXR-10": [('function', 'ADXR'), ('interval', 'daily'), ('time_period', 10)],
    "ADXR-20": [('function', 'ADXR'), ('interval', 'daily'), ('time_period', 20)],
    "ADXR-30": [('function', 'ADXR'), ('interval', 'daily'), ('time_period', 30)],
    "open-APO" : [('function', 'APO'), ('interval', 'daily'), ('series_type', 'open')],
    "close-APO" : [('function', 'APO'), ('interval', 'daily'), ('series_type', 'close')],
    "high-APO" : [('function', 'APO'), ('interval', 'daily'), ('series_type', 'high')],
    "low-APO" : [('function', 'APO'), ('interval', 'daily'), ('series_type', 'low')],
    "open-PPO" : [('function', 'PPO'), ('interval', 'daily'), ('series_type', 'open')],
    "close-PPO" : [('function', 'PPO'), ('interval', 'daily'), ('series_type', 'close')],
    "high-PPO" : [('function', 'PPO'), ('interval', 'daily'), ('series_type', 'high')],
    "low-PPO" : [('function', 'PPO'), ('interval', 'daily'), ('series_type', 'low')],
    "open-MOM-10" : [('function', 'MOM'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'open')],
    "open-MOM-20" : [('function', 'MOM'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "open-MOM-50" : [('function', 'MOM'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'open')],
    "close-MOM-10" : [('function', 'MOM'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'close')],
    "close-MOM-20" : [('function', 'MOM'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "close-MOM-50" : [('function', 'MOM'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'close')],
    "high-MOM-10" : [('function', 'MOM'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'high')],
    "high-MOM-20" : [('function', 'MOM'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "high-MOM-50" : [('function', 'MOM'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'high')],
    "low-MOM-10" : [('function', 'MOM'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'low')],
    "low-MOM-20" : [('function', 'MOM'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "low-MOM-50" : [('function', 'MOM'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'low')],
    "BOP": [('function', 'BOP'), ('interval', 'daily')],
    "CCI-10": [('function', 'CCI'), ('interval', 'daily'), ('time_period', 10)],
    "CCI-20": [('function', 'CCI'), ('interval', 'daily'), ('time_period', 20)],
    "CCI-50": [('function', 'CCI'), ('interval', 'daily'), ('time_period', 50)],
    "CCI-100": [('function', 'CCI'), ('interval', 'daily'), ('time_period', 100)],
    "open-CMO-5" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'open')],
    "open-CMO-10" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'open')],
    "open-CMO-20" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "open-CMO-30" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'open')],
    "close-CMO-5" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'close')],
    "close-CMO-10" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'close')],
    "close-CMO-20" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "close-CMO-30" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'close')],
    "high-CMO-5" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'high')],
    "high-CMO-10" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'high')],
    "high-CMO-20" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "high-CMO-30" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'high')],
    "low-CMO-5" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'low')],
    "low-CMO-10" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'low')],
    "low-CMO-20" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "low-CMO-30" : [('function', 'CMO'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'low')],
    "open-ROC-5" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'open')],
    "open-ROC-10" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'open')],
    "open-ROC-20" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "open-ROC-50" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'open')],
    "open-ROC-30" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'open')],
    "open-ROC-15" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'open')],
    "close-ROC-5" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'close')],
    "close-ROC-10" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'close')],
    "close-ROC-20" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "close-ROC-50" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'close')],
    "close-ROC-30" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'close')],
    "close-ROC-15" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'close')],
    "high-ROC-5" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'high')],
    "high-ROC-10" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'high')],
    "high-ROC-20" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "high-ROC-50" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'high')],
    "high-ROC-30" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'high')],
    "high-ROC-15" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'high')],
    "low-ROC-5" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'low')],
    "low-ROC-10" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'low')],
    "low-ROC-20" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "low-ROC-50" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'low')],
    "low-ROC-30" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'low')],
    "low-ROC-15" : [('function', 'ROC'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'low')],
    "open-ROCR-5" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'open')],
    "open-ROCR-10" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'open')],
    "open-ROCR-20" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "open-ROCR-50" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'open')],
    "open-ROCR-30" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'open')],
    "open-ROCR-15" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'open')],
    "close-ROCR-5" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'close')],
    "close-ROCR-10" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'close')],
    "close-ROCR-20" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "close-ROCR-50" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'close')],
    "close-ROCR-30" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'close')],
    "close-ROCR-15" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'close')],
    "high-ROCR-5" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'high')],
    "high-ROCR-10" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'high')],
    "high-ROCR-20" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "high-ROCR-50" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'high')],
    "high-ROCR-30" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'high')],
    "high-ROCR-15" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'high')],
    "low-ROCR-5" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'low')],
    "low-ROCR-10" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'low')],
    "low-ROCR-20" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "low-ROCR-50" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'low')],
    "low-ROCR-30" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'low')],
    "low-ROCR-15" : [('function', 'ROCR'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'low')],
    "AROON-7": [('function', 'AROON'), ('interval', 'daily'), ('time_period', 7)],
    "AROON-10": [('function', 'AROON'), ('interval', 'daily'), ('time_period', 10)],
    "AROON-20": [('function', 'AROON'), ('interval', 'daily'), ('time_period', 20)],
    "AROON-14": [('function', 'AROON'), ('interval', 'daily'), ('time_period', 14)],
    "AROON-25": [('function', 'AROON'), ('interval', 'daily'), ('time_period', 25)],
    "AROONOSC-7": [('function', 'AROONOSC'), ('interval', 'daily'), ('time_period', 7)],
    "AROONOSC-10": [('function', 'AROONOSC'), ('interval', 'daily'), ('time_period', 10)],
    "AROONOSC-20": [('function', 'AROONOSC'), ('interval', 'daily'), ('time_period', 20)],
    "AROONOSC-14": [('function', 'AROONOSC'), ('interval', 'daily'), ('time_period', 14)],
    "AROONOSC-25": [('function', 'AROONOSC'), ('interval', 'daily'), ('time_period', 25)],
    "MFI-7": [('function', 'MFI'), ('interval', 'daily'), ('time_period', 7)],
    "MFI-10": [('function', 'MFI'), ('interval', 'daily'), ('time_period', 10)],
    "MFI-20": [('function', 'MFI'), ('interval', 'daily'), ('time_period', 20)],
    "MFI-50": [('function', 'MFI'), ('interval', 'daily'), ('time_period', 50)],
    "MFI-30": [('function', 'MFI'), ('interval', 'daily'), ('time_period', 30)],
    "open-TRIX-5" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'open')],
    "open-TRIX-10" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'open')],
    "open-TRIX-20" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "open-TRIX-30" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'open')],
    "open-TRIX-25" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 25), ('series_type', 'open')],
    "open-TRIX-15" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'open')],
    "close-TRIX-5" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'close')],
    "close-TRIX-10" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'close')],
    "close-TRIX-20" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "close-TRIX-30" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'close')],
    "close-TRIX-25" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 25), ('series_type', 'close')],
    "close-TRIX-15" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'close')],
    "high-TRIX-5" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'high')],
    "high-TRIX-10" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'high')],
    "high-TRIX-20" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "high-TRIX-30" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'high')],
    "high-TRIX-25" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 25), ('series_type', 'high')],
    "high-TRIX-15" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'high')],
    "low-TRIX-5" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'low')],
    "low-TRIX-10" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'low')],
    "low-TRIX-20" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "low-TRIX-30" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'low')],
    "low-TRIX-25" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 25), ('series_type', 'low')],
    "low-TRIX-15" : [('function', 'TRIX'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'low')],
    "ULTOSC" : [('function', 'ULTOSC'), ('interval', 'daily')],
    "DX-5": [('function', 'DX'), ('interval', 'daily'), ('time_period', 5)],
    "DX-10": [('function', 'DX'), ('interval', 'daily'), ('time_period', 10)],
    "DX-20": [('function', 'DX'), ('interval', 'daily'), ('time_period', 20)],
    "DX-30": [('function', 'DX'), ('interval', 'daily'), ('time_period', 30)],
    "DX-15": [('function', 'DX'), ('interval', 'daily'), ('time_period', 15)],
    "DX-25": [('function', 'DX'), ('interval', 'daily'), ('time_period', 25)],
    "MINUS_DI-5": [('function', 'MINUS_DI'), ('interval', 'daily'), ('time_period', 5)],
    "MINUS_DI-10": [('function', 'MINUS_DI'), ('interval', 'daily'), ('time_period', 10)],
    "MINUS_DI-20": [('function', 'MINUS_DI'), ('interval', 'daily'), ('time_period', 20)],
    "MINUS_DI-30": [('function', 'MINUS_DI'), ('interval', 'daily'), ('time_period', 30)],
    "MINUS_DI-15": [('function', 'MINUS_DI'), ('interval', 'daily'), ('time_period', 15)],
    "MINUS_DI-25": [('function', 'MINUS_DI'), ('interval', 'daily'), ('time_period', 25)],
    "PLUS_DI-5": [('function', 'PLUS_DI'), ('interval', 'daily'), ('time_period', 5)],
    "PLUS_DI-10": [('function', 'PLUS_DI'), ('interval', 'daily'), ('time_period', 10)],
    "PLUS_DI-20": [('function', 'PLUS_DI'), ('interval', 'daily'), ('time_period', 20)],
    "PLUS_DI-30": [('function', 'PLUS_DI'), ('interval', 'daily'), ('time_period', 30)],
    "PLUS_DI-15": [('function', 'PLUS_DI'), ('interval', 'daily'), ('time_period', 15)],
    "PLUS_DI-25": [('function', 'PLUS_DI'), ('interval', 'daily'), ('time_period', 25)],
    "MINUS_DM-5": [('function', 'MINUS_DM'), ('interval', 'daily'), ('time_period', 5)],
    "MINUS_DM-10": [('function', 'MINUS_DM'), ('interval', 'daily'), ('time_period', 10)],
    "MINUS_DM-20": [('function', 'MINUS_DM'), ('interval', 'daily'), ('time_period', 20)],
    "MINUS_DM-30": [('function', 'MINUS_DM'), ('interval', 'daily'), ('time_period', 30)],
    "MINUS_DM-15": [('function', 'MINUS_DM'), ('interval', 'daily'), ('time_period', 15)],
    "MINUS_DM-25": [('function', 'MINUS_DM'), ('interval', 'daily'), ('time_period', 25)],
    "PLUS_DM-5": [('function', 'PLUS_DM'), ('interval', 'daily'), ('time_period', 5)],
    "PLUS_DM-10": [('function', 'PLUS_DM'), ('interval', 'daily'), ('time_period', 10)],
    "PLUS_DM-20": [('function', 'PLUS_DM'), ('interval', 'daily'), ('time_period', 20)],
    "PLUS_DM-30": [('function', 'PLUS_DM'), ('interval', 'daily'), ('time_period', 30)],
    "PLUS_DM-15": [('function', 'PLUS_DM'), ('interval', 'daily'), ('time_period', 15)],
    "PLUS_DM-25": [('function', 'PLUS_DM'), ('interval', 'daily'), ('time_period', 25)],
    "open-BBANDS-10" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'open')],
    "open-BBANDS-20" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "open-BBANDS-50" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'open')],
    "open-BBANDS-15" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'open')],
    "open-BBANDS-30" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'open')],
    "open-BBANDS-50" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'open')],
    "close-BBANDS-10" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'close')],
    "close-BBANDS-20" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "close-BBANDS-50" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'close')],
    "close-BBANDS-15" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'close')],
    "close-BBANDS-30" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'close')],
    "close-BBANDS-50" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'close')],
    "high-BBANDS-10" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'high')],
    "high-BBANDS-20" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "high-BBANDS-50" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'high')],
    "high-BBANDS-15" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'high')],
    "high-BBANDS-30" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'high')],
    "high-BBANDS-50" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'high')],
    "low-BBANDS-10" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'low')],
    "low-BBANDS-20" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "low-BBANDS-50" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'low')],
    "low-BBANDS-15" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 15), ('series_type', 'low')],
    "low-BBANDS-30" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'low')],
    "low-BBANDS-50" : [('function', 'BBANDS'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'low')],
    "open-MIDPOINT-5" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'open')],
    "open-MIDPOINT-10" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'open')],
    "open-MIDPOINT-20" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'open')],
    "open-MIDPOINT-50" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'open')],
    "open-MIDPOINT-30" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'open')],
    "close-MIDPOINT-5" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'close')],
    "close-MIDPOINT-10" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'close')],
    "close-MIDPOINT-20" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'close')],
    "close-MIDPOINT-50" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'close')],
    "close-MIDPOINT-30" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'close')],
    "high-MIDPOINT-5" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'high')],
    "high-MIDPOINT-10" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'high')],
    "high-MIDPOINT-20" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'high')],
    "high-MIDPOINT-50" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'high')],
    "high-MIDPOINT-30" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'high')],
    "low-MIDPOINT-5" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 5), ('series_type', 'low')],
    "low-MIDPOINT-10" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 10), ('series_type', 'low')],
    "low-MIDPOINT-20" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 20), ('series_type', 'low')],
    "low-MIDPOINT-50" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 50), ('series_type', 'low')],
    "low-MIDPOINT-30" : [('function', 'MIDPOINT'), ('interval', 'daily'), ('time_period', 30), ('series_type', 'low')],
    "MIDPRICE-5": [('function', 'MIDPRICE'), ('interval', 'daily'), ('time_period', 5)],
    "MIDPRICE-10": [('function', 'MIDPRICE'), ('interval', 'daily'), ('time_period', 10)],
    "MIDPRICE-20": [('function', 'MIDPRICE'), ('interval', 'daily'), ('time_period', 20)],
    "MIDPRICE-50": [('function', 'MIDPRICE'), ('interval', 'daily'), ('time_period', 50)],
    "MIDPRICE-100": [('function', 'MIDPRICE'), ('interval', 'daily'), ('time_period', 100)],
    "MIDPRICE-30": [('function', 'MIDPRICE'), ('interval', 'daily'), ('time_period', 30)],
    "SAR": [('function', 'SAR'), ('interval', 'daily')],
    "TRANGE": [('function', 'TRANGE'), ('interval', 'daily')],
    "NATR-5": [('function', 'NATR'), ('interval', 'daily'), ('time_period', 5)],
    "NATR-10": [('function', 'NATR'), ('interval', 'daily'), ('time_period', 10)],
    "NATR-20": [('function', 'NATR'), ('interval', 'daily'), ('time_period', 20)],
    "NATR-50": [('function', 'NATR'), ('interval', 'daily'), ('time_period', 50)],
    "NATR-100": [('function', 'NATR'), ('interval', 'daily'), ('time_period', 100)],
    "NATR-30": [('function', 'NATR'), ('interval', 'daily'), ('time_period', 30)],
    "AD" : [('function', 'AD'), ('interval', 'daily')],
    "ADOSC" : [('function', 'ADOSC'), ('interval', 'daily')],
    "OBV" : [('function', 'OBV'), ('interval', 'daily')],
    "open-HT_TRENDLINE" : [('function', 'HT_TRENDLINE'), ('interval', 'daily'), ('series_type', 'open')],
    "close-HT_TRENDLINE" : [('function', 'HT_TRENDLINE'), ('interval', 'daily'), ('series_type', 'close')],
    "high-HT_TRENDLINE" : [('function', 'HT_TRENDLINE'), ('interval', 'daily'), ('series_type', 'high')],
    "low-HT_TRENDLINE" : [('function', 'HT_TRENDLINE'), ('interval', 'daily'), ('series_type', 'low')],
    "open-HT_SINE" : [('function', 'HT_SINE'), ('interval', 'daily'), ('series_type', 'open')],
    "close-HT_SINE" : [('function', 'HT_SINE'), ('interval', 'daily'), ('series_type', 'close')],
    "high-HT_SINE" : [('function', 'HT_SINE'), ('interval', 'daily'), ('series_type', 'high')],
    "low-HT_SINE" : [('function', 'HT_SINE'), ('interval', 'daily'), ('series_type', 'low')],
    "open-HT_TRENDMODE" : [('function', 'HT_TRENDMODE'), ('interval', 'daily'), ('series_type', 'open')],
    "close-HT_TRENDMODE" : [('function', 'HT_TRENDMODE'), ('interval', 'daily'), ('series_type', 'close')],
    "high-HT_TRENDMODE" : [('function', 'HT_TRENDMODE'), ('interval', 'daily'), ('series_type', 'high')],
    "low-HT_TRENDMODE" : [('function', 'HT_TRENDMODE'), ('interval', 'daily'), ('series_type', 'low')],
    "open-HT_DCPERIOD" : [('function', 'HT_DCPERIOD'), ('interval', 'daily'), ('series_type', 'open')],
    "close-HT_DCPERIOD" : [('function', 'HT_DCPERIOD'), ('interval', 'daily'), ('series_type', 'close')],
    "high-HT_DCPERIOD" : [('function', 'HT_DCPERIOD'), ('interval', 'daily'), ('series_type', 'high')],
    "low-HT_DCPERIOD" : [('function', 'HT_DCPERIOD'), ('interval', 'daily'), ('series_type', 'low')],
    "open-HT_DCPHASE" : [('function', 'HT_DCPHASE'), ('interval', 'daily'), ('series_type', 'open')],
    "close-HT_DCPHASE" : [('function', 'HT_DCPHASE'), ('interval', 'daily'), ('series_type', 'close')],
    "high-HT_DCPHASE" : [('function', 'HT_DCPHASE'), ('interval', 'daily'), ('series_type', 'high')],
    "low-HT_DCPHASE" : [('function', 'HT_DCPHASE'), ('interval', 'daily'), ('series_type', 'low')],
    "open-HT_PHASOR" : [('function', 'HT_PHASOR'), ('interval', 'daily'), ('series_type', 'open')],
    "close-HT_PHASOR" : [('function', 'HT_PHASOR'), ('interval', 'daily'), ('series_type', 'close')],
    "high-HT_PHASOR" : [('function', 'HT_PHASOR'), ('interval', 'daily'), ('series_type', 'high')],
    "low-HT_PHASOR" : [('function', 'HT_PHASOR'), ('interval', 'daily'), ('series_type', 'low')],
}

In [6]:
def get_data_for(ticker):
    url = f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={ticker}&outputsize=full&datatype=csv&apikey={API_KEY}"
    request.urlretrieve(url, f"RAW-{ticker}.csv")
    df = pd.read_csv(f"RAW-{ticker}.csv")
    df = df[df["timestamp"] >= start_date]
    df = df.sort_values("timestamp")
    df = df.reset_index(drop=True)
    return df

In [7]:
def prepare_data(df):
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    df = df.set_index("timestamp")
    assert df[df['split_coefficient'] != 1].empty, "There are split coefficients different from 1"
    df = df.drop(columns=["split_coefficient", "adjusted_close"])
    
    return df

In [8]:
def get_indicator_data(ticker, col_name, params):
    while True:
            query_params = '&'.join([f"{k}={v}" for k, v in params])
            url = f"https://www.alphavantage.co/query?symbol={ticker}&apikey={API_KEY}&datatype=csv&{query_params}"
            request.urlretrieve(url, f"indicator.csv")
            
            try:
                with open(f"indicator.csv", "r") as f:
                    first_line = f.readline().strip()
                    if first_line == "{":
                        raise Exception("API limit reached")
                    elif first_line == "{}":
                        raise Exception("API call returned empty response")
            except Exception as e:
                if "API limit reached" in str(e):
                    print(f"[bright_red]API limit reached for {ticker} - {col_name}")
                    time.sleep(10)
                    continue
                print(f"[bright_red]Error occured while getting data for {ticker} - {col_name}: {e}")
                return None
            
            df = pd.read_csv(f"indicator.csv")
            df = df[df["time"] >= start_date]
            df = df.sort_values("time")
            df = df.reset_index(drop=True)
            df["time"] = pd.to_datetime(df["time"])
            df = df.set_index("time")
            for col in df.columns:
                if col != "time":
                    df = df.rename(columns={col: f"{col_name}-{col}"})
            break
    return df

In [9]:
def get_data_with_indicators(ticker, progress=None):
    df = get_data_for(ticker)
    df = prepare_data(df)
    if progress is not None:
        indicator_task = progress.add_task("Getting indicators", total=len(indicators))
    for indicator, params in indicators.items():
        data = get_indicator_data(ticker, indicator, params)
        if data is not None:
            df = pd.concat([df, data], axis=1)
        if progress is not None:
            progress.update(indicator_task, advance=1)
    return df

In [10]:
with Progress(SpinnerColumn(),*Progress.get_default_columns(),TimeElapsedColumn(),expand=True) as progress:
    sector_task = progress.add_task("Getting data for sectors", total=len(sectors) + 1)
    
    df = get_data_with_indicators(sp500, progress)
    df.to_csv(f"{sp500}.csv")
    del df
    progress.update(sector_task, advance=1)
    
    for sector in sectors:
        df = get_data_with_indicators(sector, progress)
        df.to_csv(f"{sector}.csv")
        del df
        progress.update(sector_task, advance=1)

Output()

API limit reached for XLC - close-KAMA-5

API limit reached for XLC - close-KAMA-5

API limit reached for XLC - AROONOSC-10

API limit reached for XLC - AROONOSC-10

...

API limit reached for XLRE - open-ROCR-20

API limit reached for XLRE - open-ROCR-20

API limit reached for XLRE - close-HT_PHASOR

API limit reached for XLRE - close-HT_PHASOR

API limit reached for XLU - ADXR-30

In [ ]:
def change_data_to_change_ratio(df):
    shifted_df = df.shift(1)
    df = (df - shifted_df) / shifted_df
    
    df = df.fillna(0)
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    
    return df

In [33]:
with Progress(SpinnerColumn(),*Progress.get_default_columns(),TimeElapsedColumn(),expand=True) as progress:
    sector_task = progress.add_task("Changing the data to change ratio", total=len(sectors) + 1)
    
    df = pd.read_csv(f"{sp500}.csv", index_col=0)
    df.drop(columns=["dividend_amount"], inplace=True)
    df = change_data_to_change_ratio(df)
    
    df.to_csv(f"norm-{sp500}.csv")
    progress.update(sector_task, advance=1)
    
    for sector in sectors:
        df = pd.read_csv(f"{sector}.csv", index_col=0)
        df.drop(columns=["dividend_amount"], inplace=True)
        df = change_data_to_change_ratio(df)
        
        df.to_csv(f"norm-{sector}.csv")
        progress.update(sector_task, advance=1)
    progress.refresh()

Output()